### COVID-19: Face Mask Detector with OpenCV, Keras/TensorFlow, and Deep Learning


### Two-phase COVID-19 face mask detector

![](../image/Face-Mask-Detection/face_mask_detection_phases.png)

* In order to train a custom face mask detector, we need to break our project into two distinct phases, each with its own respective sub-steps (as shown in Figure above):

1. Training: Here we’ll focus on loading our face mask detection dataset from disk, training a model (using Keras/TensorFlow) on this dataset, and then serializing the face mask detector to disk.

2. Deployment: Once the face mask detector is trained, we can then move on to loading the mask detector, performing face detection, and then classifying each face as with_mask or without_mask.

### Our COVID-19 face mask detection dataset

![](../image/Face-Mask-Detection/face_mask_detection_dataset.jpg)

#### This dataset consists of 1,376 images belonging to two classes:

* with_mask: 690 images
* without_mask: 686 images

<b>Our goal is to train a custom deep learning model to detect whether a person is or is not wearing a mask.</b>

* Facial landmarks allow us to automatically infer the location of facial structures, including:

 * Eyes
 * Eyebrows
 * Nose
 * Mouth
 * Jawline

<b>To use facial landmarks to build a dataset of faces wearing face masks, we need to first start with an image of a person not wearing a face mask:</b>
![](../image/Face-Mask-Detection/face_mask_detection_input.jpg)

<b>From there, we apply face detection to compute the bounding box location of the face in the image:</b>
![](../image/Face-Mask-Detection/face_mask_detection_face_detected.jpg)

<b>Once we know where in the image the face is, we can extract the face Region of Interest (ROI):<b>
![](../image/Face-Mask-Detection/face_mask_detection_face_extracted.jpg)
    
<b>And from there, we apply facial landmarks, allowing us to localize the eyes, nose, mouth, etc.:</b>
![](../image/Face-Mask-Detection/face_mask_detection_facial_landmarks.png)

<b>Next, we need an image of a mask (with a transparent background) such as the one below:</b>
![](../image/Face-Mask-Detection/face_mask_detection_face_augment_mask.png)

<b>This mask will be automatically applied to the face by using the facial landmarks (namely the points along the chin and nose) to compute where the mask will be placed.

The mask is then resized and rotated, placing it on the face:</b>
![](../image/Face-Mask-Detection/face_mask_detection_augmented_with_mask.jpg)

<b>We can then repeat this process for all of our input images, thereby creating our artificial face mask dataset:</b>
![](../image/Face-Mask-Detection/face_mask_detection_masks_montage.jpg)

* However, there is a caveat you should be aware of when using this method to artificially create a dataset!

 1. If you use a set of images to create an artificial dataset of people wearing masks, you cannot “re-use” the images without  masks in your training set — you still need to gather non-face mask images that were not used in the artificial generation process!

 2. If you include the original images used to generate face mask samples as non-face mask samples, your model will become heavily biased and fail to generalize well. Avoid that at all costs by taking the time to gather new examples of faces without masks.

### Project Structure

├── dataset
│   ├── with_mask [690 entries]
│   └── without_mask [686 entries]
├── examples
│   ├── example_01.png
│   ├── example_02.png
│   └── example_03.png
├── face_detector
│   ├── deploy.prototxt
│   └── res10_300x300_ssd_iter_140000.caffemodel
├── detect_mask_image.py
├── detect_mask_video.py
├── mask_detector.model
├── plot.png
└── train_mask_detector.py

* Three image examples/ are provided so that you can test the static image face mask detector.

* We’ll be reviewing three Python scripts in this tutorial:

 1. train_mask_detector.py: Accepts our input dataset and fine-tunes MobileNetV2 upon it to create our mask_detector.model. A training history plot.png containing accuracy/loss curves is also produced
 2. detect_mask_image.py: Performs face mask detection in static images
 3. detect_mask_video.py: Using your webcam, this script applies face mask detection to every frame in the stream

### Implementing our COVID-19 face mask detector training script with Keras and TensorFlow

* Now that we’ve reviewed our face mask dataset, let’s learn how we can use Keras and TensorFlow to train a classifier to automatically detect whether a person is wearing a mask or not.

* To accomplish this task, we’ll be fine-tuning the MobileNet V2 architecture, a highly efficient architecture that can be applied to embedded devices with limited computational capacity (ex., Raspberry Pi, Google Coral, NVIDIA Jetson Nano, etc.).

* Deploying our face mask detector to embedded devices could reduce the cost of manufacturing such face mask detection systems, hence why we choose to use this architecture.

#### train_mask_detector.py

* Our set of tensorflow.keras imports allow for:

 1. Data augmentation
 2. Loading the MobilNetV2 classifier (we will fine-tune this model with pre-trained ImageNet weights)
 3. Building a new fully-connected (FC) head
 5. Pre-processing
 6. Loading image data

* We’ll use scikit-learn (sklearn) for binarizing class labels, segmenting our dataset, and printing a classification report.

* The imutils paths implementation will help us to find and list images in our dataset. And we’ll use matplotlib to plot our training curves.

In [1]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

* The command line arguments include:

 1. --dataset: The path to the input dataset of faces and and faces with masks
 2. --plot: The path to your output training history plot, which will be generated using matplotlib
 3. --model: The path to the resulting serialized face mask classification model

* Define the deep learning hyperparameters in one place:
  Specify the hyperparameter constants including the initial learning rate(INIT_LR), number of training epochs, and batch size.

In [2]:
# construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-d", "--dataset", required=True,
#     help="path to input dataset")
# ap.add_argument("-p", "--plot", type=str, default="plot.png",
#     help="path to output loss/accuracy plot")
# ap.add_argument("-m", "--model", type=str,
#     default="mask_detector.model",
#     help="path to output face mask detector model")
# args = vars(ap.parse_args())

dataset = "D:/Python/pyimagesearch-Opencv-tutorials/covid-19-face-mask-detector-with-opencv-keras-tensorflow-and-deep-learning/face-mask-dataset/dataset/train"
plot = "D:/Python/pyimagesearch-Opencv-tutorials/covid-19-face-mask-detector-with-opencv-keras-tensorflow-and-deep-learning/face_detector/plot.png"
model = "D:/Python/pyimagesearch-Opencv-tutorials/covid-19-face-mask-detector-with-opencv-keras-tensorflow-and-deep-learning/face_detector/mask_detector.model"
    
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

* Load and pre-process the training data:

 1. Grab all of the imagePaths in the dataset.
 2. Initialize data and labels lists.
 3. Loop over the imagePaths and loading + pre-processing images. Pre-processing steps include resizing to 224×224 pixels, conversion to array format, and scaling the pixel intensities in the input image to the range [-1, 1] (via the preprocess_input convenience function)
 4. Append the pre-processed image and associated label to the data and labels lists, respectively.
 5. Ensure the training data is in NumPy array format.
 6. We assume that the entire dataset is small enough to fit into memory. If the dataset is larger than the memory available, we suggest using HDF5.

* Next, we’ll encode our labels, partition our dataset, and prepare for data augmentation:

* One-hot encode our class labels, meaning that our data will be in the following format:

COVID-19: Face Mask Detector with OpenCV, Keras/TensorFlow, and Deep Learning
$ python  train_mask_detector.py --dataset  dataset 
[INFO] loading images...
-> (trainX, testX, trainY, testY) = train_test_split(data, labels,
(Pdb) labels[500:]
array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)
(Pdb)

* As you can see, each element of our labels array consists of an array in which only one index is “hot” (i.e., 1).

* Using scikit-learn’s convenience method, segment our data into 80% training and the remaining 20% for testing.

* During training, we’ll be applying on-the-fly mutations to our images in an effort to improve generalization. This is known as  data augmentation, where the random rotation, zoom, shear, shift, and flip parameters are established. We’ll use the aug object at training time.

* But first, we need to prepare MobileNetV2 for fine-tuning:

* Fine-tuning setup is a three-step process:

 1. Load MobileNet with pre-trained ImageNet weights, leaving off head of network.
 2. Construct a new FC head, and append it to the base in place of the old head.
 2. Freeze the base layers of the network. The weights of these base layers will not be updated during the process of backpropagation, whereas the head layer weights will be tuned.

* Fine-tuning is a strategy, which is nearly always recommend to establish a baseline model while saving considerable time.

* With the data prepared and model architecture in place for fine-tuning, we’re now ready to compile and train our face mask detector network:

 1. Compile the model with the Adam optimizer, a learning rate decay schedule, and binary cross-entropy. If you’re building from this training script with > 2 classes, be sure to use categorical cross-entropy.

 2. Face mask training is launched. Notice how our data augmentation object (aug) will be providing batches of mutated image data.

* Once training is complete, we’ll evaluate the resulting model on the test set:
 
 1. Here, make predictions on the test set, grabbing the highest probability class label indices. Then, we print a classification report in the terminal for inspection.

2. Serializes our face mask classification model to disk.

* The last step is to plot the accuracy and loss curves:
* Once the plot is ready, save the figure to disk using the --plot filepath.

In [ ]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
#imagePaths = list(paths.list_images(args["dataset"]))
imagePaths = list(paths.list_images(dataset))
#imagePaths = list(paths.list_images("D:/Python/pyimagesearch-Opencv-tutorials/covid-19-face-mask-detector-with-opencv-keras-tensorflow-and-deep-learning/face-mask-dataset/dataset/train"))
data = []
labels = []
# loop over the image paths
for imagePath in imagePaths:
    # extract the class label from the filename
    print("imagePath =  ", imagePath)
    label = imagePath.split(os.path.sep)[-2]
    print("label =  ", label)
    
    # load the input image (224x224) and preprocess it
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)

# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=0.20, stratify=labels, random_state=42)
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False
    
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])

# train the head of the network
print("[INFO] training head...")
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
    target_names=lb.classes_))

# serialize the model to disk
print("[INFO] saving mask detector model...")
# model.save(args["model"], save_format="h5")
model.save(model, save_format="h5")

# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
#plt.savefig(args["plot"])
plt.savefig(plot)

### Training the COVID-19 face mask detector with Keras/TensorFlow

* We are now ready to train our face mask detector using Keras, TensorFlow, and Deep Learning.
* Open up a terminal, and execute the following command:<br>
 $ python train_mask_detector.py --dataset dataset
 
* As you can see, we are obtaining ~99% accuracy on our test set.

* Looking at COVID-19 face mask detector training accuracy/loss curves, we can see that there are little signs of overfitting, with the validation loss lower than the training loss.

* Given these results, we are hopeful that our model will generalize well to images outside our training and testing set.

### Implementing our COVID-19 face mask detector for images with OpenCV

* Now that our face mask detector is trained, let’s learn how we can:

 1. Load an input image from disk
 2. Detect faces in the image
 3. Apply our face mask detector to classify the face as either with_mask or without_mask
 
* detect_mask_image.py

 * Our driver script requires three TensorFlow/Keras imports to (1) load our MaskNet model and (2) pre-process the input image.

 * OpenCV is required for display and image manipulations.

In [4]:
# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import cv2
import os

* The next step is to parse command line arguments:
* Our four command line arguments include:

 * --image: The path to the input image containing faces for inference
 * --face: The path to the face detector model directory (we need to localize faces prior to classifying them)
 * --model: The path to the face mask detector model that we trained earlier in this tutorial
 * --confidence: An optional probability threshold can be set to override 50% to filter weak face detections

In [5]:
# # construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required=True,
#     help="path to input image")
# ap.add_argument("-f", "--face", type=str,
#     default="face_detector",
#     help="path to face detector model directory")
# ap.add_argument("-m", "--model", type=str,
#     default="mask_detector.model",
#     help="path to trained face mask detector model")
# ap.add_argument("-c", "--confidence", type=float, default=0.5,
#     help="minimum probability to filter weak detections")
# args = vars(ap.parse_args())

* Next, we’ll load both our face detector and face mask classifier models:

In [6]:
default_confidence = 0.5
# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
#prototxtPath = os.path.sep.join([args["face"], "deploy.prototxt"])
prototxtPath = "D:/Python/pyimagesearch-Opencv-tutorials/covid-19-face-mask-detector-with-opencv-keras-tensorflow-and-deep-learning/face_detector/deploy.prototxt"
#weightsPath = os.path.sep.join([args["face"], "res10_300x300_ssd_iter_140000.caffemodel"])
weightsPath = "D:/Python/pyimagesearch-Opencv-tutorials/covid-19-face-mask-detector-with-opencv-keras-tensorflow-and-deep-learning/face_detector/res10_300x300_ssd_iter_140000.caffemodel"
net = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask detector model from disk
print("[INFO] loading face mask detector model...")
# model = load_model(args["model"])
model = load_model("D:/Python/pyimagesearch-Opencv-tutorials/covid-19-face-mask-detector-with-opencv-keras-tensorflow-and-deep-learning/face_detector/mask_detector.model")

[INFO] loading face detector model...
[INFO] loading face mask detector model...


* With our deep learning models now in memory, our next step is to load and pre-process an input image:

In [7]:
# load the input image from disk, clone it, and grab the image spatial dimensions
#image = cv2.imread(args["image"])
#image = cv2.imread("D:/Python/pyimagesearch-Opencv-tutorials/covid-19-face-mask-detector-with-opencv-keras-tensorflow-and-deep-learning/face-mask-dataset/dataset/test/with_mask/480-with-mask.jpg")
image = cv2.imread("D:/Python/pyimagesearch-Opencv-tutorials/covid-19-face-mask-detector-with-opencv-keras-tensorflow-and-deep-learning/face-mask-dataset/dataset/test/without_mask/409.jpg")
orig = image.copy()
(h, w) = image.shape[:2]

# construct a blob from the image
blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))

# pass the blob through the network and obtain the face detections
print("[INFO] computing face detections...")
net.setInput(blob)
detections = net.forward()

[INFO] computing face detections...


* Upon loading our --image from disk, we make a copy and grab frame dimensions for future scaling and display purposes.

* Pre-processing is handled by OpenCV’s blobFromImage function. As shown in the parameters, we resize to 300×300 pixels and  perform mean subtraction.

* Perform face detection to localize where in the image all faces are.

* Once we know where each face is predicted to be, we’ll ensure they meet the --confidence threshold before we extract the faceROIs:

 * Here, we loop over our detections and extract the confidence to measure against the --confidence threshold.

 * We then compute bounding box value for a particular face and ensure that the box falls within the boundaries of the image.
 
* Next, we’ll run the face ROI through our MaskNet model:

* In this block, we:

 1. Extract the face ROI via NumPy slicing.
 2. Pre-process the ROI the same way we did during training.
 3. Perform mask detection to predict with_mask or without_mask.
 4. From here, we will annotate and display the result!
 
* First, we determine the class label based on probabilities returned by the mask detector model and assign an associated color for the annotation. The color will be “green” for with_mask and “red” for without_mask.

* We then draw the label text (including class and probability), as well as a bounding box rectangle for the face, using OpenCV drawing functions.

* Once all detections have been processed, display the output image.

In [8]:
# loop over the detections
for i in range(0, detections.shape[2]):
    # extract the confidence (i.e., probability) associated with the detection
    confidence = detections[0, 0, i, 2]
    # filter out weak detections by ensuring the confidence is greater than the minimum confidence
    #if confidence > args["confidence"]:
    if confidence > default_confidence:
        # compute the (x, y)-coordinates of the bounding box for the object
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        # ensure the bounding boxes fall within the dimensions of the frame
        (startX, startY) = (max(0, startX), max(0, startY))
        (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
        
        # extract the face ROI, convert it from BGR to RGB channel ordering, resize it to 224x224, and preprocess it
        face = image[startY:endY, startX:endX]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)
        # pass the face through the model to determine if the face has a mask or not
        (mask, withoutMask) = model.predict(face)[0]
        
        # determine the class label and color we'll use to draw the bounding box and text
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        # include the probability in the label
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        
        # display the label and bounding box rectangle on the output frame
        cv2.putText(image, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)

# show the output image
cv2.imshow("Output", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### COVID-19 face mask detection in images with OpenCV

* Let’s put our COVID-19 face mask detector to work!

* Open up a terminal, and execute the following command:<br>

* $ python detect_mask_image.py --image examples/example_01.png

### Implementing our COVID-19 face mask detector in real-time video streams with OpenCV

* detect_mask_video.py

In [16]:
# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
import os

* The algorithm for this script is the same, but it is pieced together in such a way to allow for processing every frame of your webcam stream.

* Thus, the only difference when it comes to imports is that we need a VideoStream class and time. Both of these will help us to work with the stream. We’ll also take advantage of imutils for its aspect-aware resizing method.

* Our face detection/mask prediction logic for this script is in the <b>detect_and_predict_mask</b> function:

* By defining this convenience function here, our frame processing loop will be a little easier to read later.

* This function detects faces and then applies our face mask classifier to each face ROI. Such a function consolidates our code — it could even be moved to a separate Python file if you so choose.

* Our <b>detect_and_predict_mask</b> function accepts three parameters:

 * frame: A frame from our stream
 * faceNet: The model used to detect where in the image faces are
 * maskNet: Our COVID-19 face mask classifier model

* Inside, we construct a blob, detect faces, and initialize lists, two of which the function is set to return. These lists include our faces (i.e., ROIs), locs (the face locations), and preds (the list of mask/no mask predictions).

* From here, we’ll loop over the face detections:
 * Inside the loop, we filter out weak detections and extract bounding boxes while ensuring bounding box coordinates do not fall outside the bounds of the image.

* Next, we’ll add face ROIs to two of our corresponding lists:

* After extracting face ROIs and pre-processing, we append the the face ROIs and bounding boxes to their respective lists.

* We’re now ready to run our faces through our mask predictor:

* The logic here is built for speed. First we ensure at least one face was detected — if not, we’ll return empty preds.

* Secondly, we are performing inference on our entire batch of faces in the frame so that our pipeline is faster. It wouldn’t make sense to write another loop to make predictions on each face individually due to the overhead (especially if you are using a GPU that requires a lot of overhead communication on your system bus). It is more efficient to perform predictions in batch.

* Finally, it returns our face bounding box locations and corresponding mask/not mask predictions to the caller.

In [17]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    # grab the dimensions of the frame and then construct a blob from it
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
        (104.0, 177.0, 123.0))
    
    # pass the blob through the network and obtain the face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()
    # initialize our list of faces, their corresponding locations, and the list of predictions from our face mask network
    faces = []
    locs = []
    preds = []
    
    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the detection
        confidence = detections[0, 0, i, 2]
        # filter out weak detections by ensuring the confidence is greater than the minimum confidence
        #if confidence > args["confidence"]:
        if confidence > default_confidence:
            # compute the (x, y)-coordinates of the bounding box for the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            
            # ensure the bounding boxes fall within the dimensions of the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            
            # extract the face ROI, convert it from BGR to RGB channel
            # ordering, resize it to 224x224, and preprocess it
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            
            # add the face and bounding boxes to their respective lists
            faces.append(face)
            locs.append((startX, startY, endX, endY))
    
    # only make a predictions if at least one face was detected
    if len(faces) > 0:
        # for faster inference we'll make batch predictions on *all*
        # faces at the same time rather than one-by-one predictions in the above `for` loop
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)
        
    # return a 2-tuple of the face locations and their corresponding locations
    return (locs, preds)

* Next, we’ll define our command line arguments:

* Our command line arguments include:

 * --face: The path to the face detector directory
 * --model: The path to our trained face mask classifier
 * --confidence: The minimum probability threshold to filter weak face detections

* With our imports, convenience function, and command line args ready to go, we just have a few initializations to handle before we loop over frames:

In [11]:
# construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-f", "--face", type=str,
#     default="face_detector",
#     help="path to face detector model directory")
# ap.add_argument("-m", "--model", type=str,
#     default="mask_detector.model",
#     help="path to trained face mask detector model")
# ap.add_argument("-c", "--confidence", type=float, default=0.5,
#     help="minimum probability to filter weak detections")
# args = vars(ap.parse_args())

In [18]:
default_confidence = 0.5
# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
# prototxtPath = os.path.sep.join([args["face"], "deploy.prototxt"])
prototxtPath = "D:/Python/pyimagesearch-Opencv-tutorials/covid-19-face-mask-detector-with-opencv-keras-tensorflow-and-deep-learning/face_detector/deploy.prototxt"
# weightsPath = os.path.sep.join([args["face"], "res10_300x300_ssd_iter_140000.caffemodel"])
weightsPath = "D:/Python/pyimagesearch-Opencv-tutorials/covid-19-face-mask-detector-with-opencv-keras-tensorflow-and-deep-learning/face_detector/res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask detector model from disk
print("[INFO] loading face mask detector model...")
# maskNet = load_model(args["model"])
maskNet = load_model("D:/Python/pyimagesearch-Opencv-tutorials/covid-19-face-mask-detector-with-opencv-keras-tensorflow-and-deep-learning/face_detector/mask_detector.model")

# initialize the video stream and allow the camera sensor to warm up
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)

[INFO] loading face detector model...
[INFO] loading face mask detector model...
[INFO] starting video stream...


* Here we have initialized our:

 * Face detector
 * COVID-19 face mask detector
 * Webcam video stream

* Let’s proceed to loop over frames in the stream:

* We begin looping over frames. Inside, we grab a frame from the stream and resize it.

* From there, we put our convenience utility to use; Then detect and predict whether people are wearing their masks or not.

* Let’s post-process (i.e., annotate) the COVID-19 face mask detection results:

* Inside our loop over the prediction results, we:

 * Unpack a face bounding box and mask/not mask prediction
 * Determine the label and color
 * Annotate the label and face bounding box

* Finally, we display the results and perform cleanup:

* After the frame is displayed, we capture key presses. If the user presses q (quit), we break out of the loop and perform housekeeping.

In [19]:
# loop over the frames from the video stream
while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=400)
    # detect faces in the frame and determine if they are wearing a
    # face mask or not
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
    
    # loop over the detected face locations and their corresponding locations
    for (box, pred) in zip(locs, preds):
        # unpack the bounding box and predictions
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred
        # determine the class label and color we'll use to draw the bounding box and text
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        # include the probability in the label
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        # display the label and bounding box rectangle on the output frame
        cv2.putText(frame, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        
    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stream.release()
vs.stop()

### Detecting COVID-19 face masks with OpenCV in real-time

* We can launch the mask detector in real-time video streams using the following command:<br>
$ python detect_mask_video.py